In [154]:
from PIL import Image, ImageDraw, ImageFont
import os,glob
import numpy as np
import random

import cv2

charactersList=['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','0','1','2','3','4','5','6','7','8','9']
fontTypes=[]

path = r'pics\white.png'
image = cv2.imread(path)
h,w,o = image.shape

# text = 'dec'
font = cv2.FONT_HERSHEY_SIMPLEX
colorlist=[]
for i in range(0,255,60):
    for j in range(0,255,60):
        for k in range(0,255,60):
            if i+j+k<250:
                colorlist.append((i,j,k))
    
for text in charactersList:
    image = cv2.imread(path)
    org = (int(h/2), int(w/2))
    fontScale = 10
    for color_num,color in enumerate(colorlist):  
        for thickness in range(2,31,4):            
            image0 = cv2.putText(image, text, org, font, fontScale, 
                         color, thickness, cv2.LINE_AA, False)
            name="pics\_"+str(text)+str(color_num)+str(thickness)+".jpg"
            isWritten = cv2.imwrite(name, image0)
